In [ ]:
# Importing dataset
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

In [ ]:
df_violence = pd.read_csv('/content/gdrive/My Drive/Kids-In-Mind/Kids_in_mind_A_B_C.csv')

In [ ]:
df1 = pd.read_csv("/content/gdrive/My Drive/Kids-In-Mind/subtitles2.csv")
df2 = pd.read_csv("/content/gdrive/My Drive/Kids-In-Mind/subtitles3.csv")
df_language = pd.concat([df1, df2],ignore_index=True)

In [ ]:
%%capture
!pip install transformers

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
# from emoji import demojize
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
tokens = word_tokenize(str(df_language['Subtitles']))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def TextClean_Final(text):
  text =str(text)
  text = text.lower()
  #text = re.sub(r'@[a-z0-9_]\S+' , '', text)
  #text = re.sub(r'&[a-z0-9_]\S+','',text)
  #text = re.sub(r'rt[\s]+', '', text)
  text = re.sub(r'\$', '', text)
  #text = re.sub(r'rt+', '', text)
    # TO remove HTML tags
  CLEANR = re.compile('<.*?>') 
  text = re.sub(CLEANR, '', text)
  
  tokens = word_tokenize(text)
  text = tokens
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  text = [word for word in stripped if word.isalpha()]
  stop_words = set(stopwords.words('english'))
  text = [w for w in text if not w in stop_words]
  text = [lemmatizer.lemmatize(word) for word in text]
  return ','.join(text)

In [ ]:
df_language['Subtitle_Clean'] = df_language['Subtitles'].apply(TextClean_Final)

In [ ]:
# Violence
X_train, X_test , y_train, y_test = train_test_split(df_violence['Violence_Gore_Description'],df_violence['Violence_Gore'],test_size=0.2,random_state=123)

In [ ]:
# Language
X_train0, X_test0 , y_train0, y_test0 = train_test_split(df_language['Subtitle_Clean'],df_language['Language'],test_size=0.2,random_state=123)

#BERT

In [ ]:
from collections import Counter
counter = Counter(df['Violence_Gore'])
for k,v in counter.items():
	per = v / len(df['Violence_Gore']) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=4, n=765 (16.225%)
Class=5, n=599 (12.704%)
Class=3, n=871 (18.473%)
Class=2, n=448 (9.502%)
Class=6, n=636 (13.489%)
Class=8, n=348 (7.381%)
Class=7, n=545 (11.559%)
Class=1, n=182 (3.860%)
Class=0, n=48 (1.018%)
Class=9, n=166 (3.521%)
Class=10, n=107 (2.269%)


In [ ]:
X=list(df['Violence_Gore_Description'])
y=list(df['Violence_Gore'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
import tensorflow as tf

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU

#BERT Training

In [ ]:
%%capture
!pip install tensorflow_addons

In [ ]:
import tensorflow_addons as tfa

In [ ]:
lr=2e-05
optimizer = tfa.optimizers.AdamW(learning_rate=lr,weight_decay=0)
# optimizer =tf.keras.optimizers.SGD(learning_rate=0.1)

In [ ]:
%%capture
!pip install transformers

In [ ]:
with tpu_strategy.scope():
  from transformers import BertTokenizer, TFBertModel

  import tensorflow as tf

  tokenizer1 = BertTokenizer.from_pretrained("bert-large-uncased")

  model1 = TFBertModel.from_pretrained("bert-large-uncased")

  input_ids=tf.keras.Input(shape=(512),dtype=tf.int32)
  token_type_ids=tf.keras.Input(shape=(512),dtype=tf.int32)
  attention_mask=tf.keras.Input(shape=(512),dtype=tf.int32)

  layer1=model1({'input_ids':input_ids,
  'token_type_ids':token_type_ids,
  'attention_mask':attention_mask})

  output1=tf.math.reduce_sum(layer1.last_hidden_state,axis=1)/layer1.last_hidden_state.shape[1]
  dense1=tf.keras.layers.Dense(1,activation='relu')(output1) #tanh
  
  output2=tf.math.reduce_sum(layer1.last_hidden_state,axis=1)/layer1.last_hidden_state.shape[1]
  dense2=tf.keras.layers.Dense(1,activation='relu')(output2) #tanh
  # dense1=tf.keras.layers.LeakyReLU(alpha=0.3)(dense1)

  model3=tf.keras.Model(inputs=[input_ids,token_type_ids,attention_mask],outputs=[dense1,dense2])

  # opt=tf.keras.optimizers.Adam(learning_rate=0.001)
  model3.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer=optimizer)
  # dense2=tf.keras.layers.Dense(1)(dense1)

In [ ]:
inputs = tokenizer1(X_train, return_tensors="tf",padding='max_length',truncation=True)
test_inputs = tokenizer1(X_test, return_tensors="tf",padding='max_length',truncation=True)

In [ ]:
# Add for language
ip1=inputs['input_ids']
ip2=inputs['token_type_ids']
ip3=inputs['attention_mask']
ip1t=test_inputs['input_ids']
ip2t=test_inputs['token_type_ids']
ip3t=test_inputs['attention_mask']

In [ ]:
# add Array of language and violence rating
y1=tf.stack(y_train)

In [ ]:
callback1 = tf.keras.callbacks.ModelCheckpoint(
    filepath='./bertviolence.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1,save_best_only=True)

In [ ]:
y_test1=tf.stack(y_test)

In [ ]:
# model3.summary()

In [ ]:
history1=model3.fit([ip1,ip2,ip3],y1,epochs=100,batch_size=64,validation_data=([ip1t,ip2t,ip3t],y_test1),callbacks=[callback1])

Epoch 1/100


59/59 [==============================] - ETA: 0s - loss: 0.9389
Epoch 1: val_loss improved from inf to 0.49063, saving model to ./bertviolence.h5
59/59 [==============================] - 139s 661ms/step - loss: 0.9389 - val_loss: 0.4906
Epoch 2/100
59/59 [==============================] - ETA: 0s - loss: 0.4157
Epoch 2: val_loss improved from 0.49063 to 0.37511, saving model to ./bertviolence.h5
59/59 [==============================] - 20s 332ms/step - loss: 0.4157 - val_loss: 0.3751
Epoch 3/100
59/59 [==============================] - ETA: 0s - loss: 0.3170
Epoch 3: val_loss improved from 0.37511 to 0.35828, saving model to ./bertviolence.h5
59/59 [==============================] - 19s 329ms/step - loss: 0.3170 - val_loss: 0.3583
Epoch 4/100
59/59 [==============================] - ETA: 0s - loss: 0.2724
Epoch 4: val_loss did not improve from 0.35828
59/59 [==============================] - 17s 291ms/step - loss: 0.2724 - val_loss: 0.3715
Epoch 5/100
59/59 [===========================

In [ ]:
preds1=model3.predict([ip1t,ip2t,ip3t])

In [ ]:
y_test1

In [ ]:
preds1

#BERT Validation

In [ ]:
# Validation
df_val = pd.read_csv('/content/gdrive/My Drive/Kids-In-Mind/caption_predcitions_s_teams.csv')
def TexttoStr_Final(text):
  text =str(text)
  return text
df_val['caption'] = df_val['caption'].apply(TexttoStr_Final)

In [ ]:
validation_inputs = tokenizer1(list(df_val['caption']),return_tensors="tf",padding='max_length',truncation=True)

In [ ]:
ip1val=validation_inputs['input_ids']
ip2val=validation_inputs['token_type_ids']
ip3val=validation_inputs['attention_mask']

In [ ]:
preds2=model3.predict([ip1val,ip2val,ip3val])

In [ ]:
preds2

array([[0.7619224],
       [1.3372272],
       [1.0204912],
       [1.0999912]], dtype=float32)

In [ ]:
# Oversamppling
# Case study of any one channel

In [ ]:
# y_test

In [ ]:
# len(preds1)

In [ ]:
# model3.summary()

In [ ]:
# model

In [ ]:
ip1test=tf.cast(ip1t,tf.float32)
ip2test=tf.cast(ip2t,tf.float32)
ip3test=tf.cast(ip3t,tf.float32)

In [ ]:
ip1t

In [ ]:
preds1=model3.predict([ip1test,ip2test,ip3test])

In [ ]:
preds1

In [ ]:
# sarpreds.count(1)

In [ ]:
# sarpreds=[]
# for x in preds1:
#     if x[0]>0.5:
#         # sarpreds.append(1)
#     else:
#         sarpreds.append(0)

In [ ]:
# tpreds=[]
# for x in preds1:
#     tpreds.append(x[0])

In [ ]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, sarpreds)

In [ ]:
# len(model3.layers[3]._layers[0]._layers[1]._layers[0])

In [ ]:
# model3.layers[3]

In [ ]:
# for x in range(6):
#     model3.layers[3]._layers[0]._layers[1]._layers[0][x].trainable=False

In [ ]:
# model3.summary()

In [ ]:
# model3.layers[-2].output

In [ ]:
# model3.load_weights()

In [ ]:
# with tpu_strategy.scope():
#     from transformers import BertTokenizer, TFBertModel
#     ndense1=tf.keras.layers.Dense(1)(model3.layers[-2].output)
#     transfermodel=tf.keras.Model(model3.inputs,ndense1)
    
# #     opt=tf.keras.optimizers.Adam(learning_rate=0.001)
#     transfermodel.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer=optimizer)

In [ ]:
# transfermodel.summary()

In [ ]:
# layerd=tf.keras.

In [ ]:
# model1.base_model

In [ ]:
# from sklearn.metrics import mean_squared_error
# mean_squared_error(y_test,tpreds)

In [ ]:
# import scipy
# scipy.stats.pearsonr(y_test,tpreds)

In [ ]:
# from sklearn.metrics import r2_score
# r2_score(y_test,tpreds)

In [ ]:
# inputs

In [ ]:
# inputs = tokenizer(["Hello, my dog is cute",'this is '], return_tensors="tf",padding='max')

# outputs = model(inputs)

# # last_hidden_states = outputs.last_hidden_state

In [ ]:
# outputs

In [ ]:
# from transformers import BertModel, BertConfig

# # Initializing a BERT bert-base-uncased style configuration

# configuration = BertConfig()

# # Initializing a model from the bert-base-uncased style configuration

# model = BertModel(configuration)

# # Accessing the model configuration

# configuration = model.config

In [ ]:
# configuration

# Multi Task Bert Modal

In [ ]:
%%capture
!pip install m3tl

In [ ]:
problem_type_dict = {'violence': 'regression', 'language': 'regression'}

In [ ]:
import m3tl
from m3tl.preproc_decorator import preprocessing_fn
from m3tl.params import Params
from m3tl.special_tokens import TRAIN
@preprocessing_fn
def violence(params: Params, mode: str):
    "Simple example to demonstrate singe modal tuple of list return"
    if mode == TRAIN:
        toy_input = X_train
        toy_target = y_train
    else:
        toy_input = X_test
        toy_target = y_test
    return toy_input, toy_target

@preprocessing_fn
def language(params: Params, mode: str):
    "Simple example to demonstrate singe modal tuple of list return"
    if mode == TRAIN:
        toy_input = X_train0
        toy_target = y_train0
    else:
        toy_input = X_test0
        toy_target = y_test0
    return toy_input, toy_target

processing_fn_dict = {'violence': violence, 'language': language}

In [ ]:
from m3tl.params import Params
params = Params()
# specify model and its loading module
params.transformer_model_name = 'bert-base-uncased'
params.transformer_model_loading = 'TFBertModel'
# specify tokenizer and its loading module
params.transformer_tokenizer_name = 'bert-base-uncased'
params.transformer_tokenizer_loading = 'BertTokenizer'
# specify config and its loading module
params.transformer_config_name = 'bert-base-uncased'
params.transformer_config_loading = 'BertConfig'

In [ ]:
from m3tl.run_bert_multitask import train_bert_multitask, eval_bert_multitask, predict_bert_multitask
problem = 'violence|language' # The dataset is different so we are using '|' . If we want to merge then we can use '&'
params.assign_mtl_model('mmoe')
model = train_bert_multitask(
    problem=problem,
    num_epochs=100,
    problem_type_dict=problem_type_dict,
    processing_fn_dict=processing_fn_dict,
    continue_training=False,
    params=params # pass params
)

2022-08-13 07:22:19.272 | INFO     | m3tl.base_params:register_multiple_problems:543 - Adding new problem violence, problem type: regression
2022-08-13 07:22:19.276 | INFO     | m3tl.base_params:register_multiple_problems:543 - Adding new problem language, problem type: regression
2022-08-13 07:22:19.279 | WARNING  | m3tl.base_params:prepare_dir:363 - bert_config not exists. will load model from huggingface checkpoint.
2022-08-13 07:22:19.895 | INFO     | m3tl.input_fn:train_eval_input_fn:59 - sampling weights: 
2022-08-13 07:22:19.898 | INFO     | m3tl.input_fn:train_eval_input_fn:60 - {
    "violence": 0.9669315560112791,
    "language": 0.033068443988720844
}
2022-08-13 07:22:20.460 | INFO     | m3tl.input_fn:train_eval_input_fn:59 - sampling weights: 
2022-08-13 07:22:20.467 | INFO     | m3tl.input_fn:train_eval_input_fn:60 - {
    "violence": 0.9669315560112791,
    "language": 0.033068443988720844
}
2022-08-13 07:22:22.819 | CRITICAL | m3tl.base_params:update_train_steps:456 - Up

Epoch 1/100


2022-08-13 07:22:31.517 | INFO     | m3tl.utils:set_phase:478 - Setting phase to train
2022-08-13 07:22:52.561 | INFO     | m3tl.utils:set_phase:478 - Setting phase to train


223/223 [==============================] - ETA: 0s - mean_acc: -0.3925 - language_neg_mse: -9.7958 - violence_neg_mse: -28.2940 - BertMultiTaskTop/language/losses/0: 9.7958 - BertMultiTaskTop/violence/losses/0: 28.2940

2022-08-13 07:26:34.874 | INFO     | m3tl.utils:set_phase:478 - Setting phase to eval


223/223 [==============================] - 275s 1s/step - mean_acc: -0.3925 - language_neg_mse: -9.7958 - violence_neg_mse: -28.2940 - BertMultiTaskTop/language/losses/0: 9.7523 - BertMultiTaskTop/violence/losses/0: 28.2768 - val_loss: 32.3729 - val_mean_acc: 0.0000e+00 - val_language_neg_mse: -7.8341 - val_violence_neg_mse: -24.5388 - val_BertMultiTaskTop/language/losses/0: 17.6911 - val_BertMultiTaskTop/violence/losses/0: 0.0000e+00
Epoch 2/100
223/223 [==============================] - 218s 978ms/step - mean_acc: -0.9424 - language_neg_mse: -8.7292 - violence_neg_mse: -22.7507 - BertMultiTaskTop/language/losses/0: 8.6903 - BertMultiTaskTop/violence/losses/0: 22.6985
Epoch 3/100
223/223 [==============================] - 217s 973ms/step - mean_acc: -0.9348 - language_neg_mse: -6.8552 - violence_neg_mse: -16.4533 - BertMultiTaskTop/language/losses/0: 6.8246 - BertMultiTaskTop/violence/losses/0: 16.4251
Epoch 4/100
223/223 [==============================] - 217s 972ms/step - mean_acc: 

In [ ]:
eval_dict = eval_bert_multitask(problem=problem,
                    problem_type_dict=problem_type_dict, processing_fn_dict=processing_fn_dict,
                    model_dir=model.params.ckpt_dir)

In [ ]:
# Training without passing any parameters
from m3tl.run_bert_multitask import train_bert_multitask, eval_bert_multitask, predict_bert_multitask
problem = 'violence|language'
# train
model = train_bert_multitask(
    problem=problem,
    num_epochs=10,
    problem_type_dict=problem_type_dict,
    processing_fn_dict=processing_fn_dict,
    continue_training=False
)

In [ ]:
eval_dict = eval_bert_multitask(problem=problem,
                    problem_type_dict=problem_type_dict, processing_fn_dict=processing_fn_dict,
                    model_dir=model.params.ckpt_dir)

In [ ]:
eval_dict

{'BertMultiTaskTop/language/losses/0': 0.061159197241067886,
 'BertMultiTaskTop/violence/losses/0': 2.725895881652832,
 'language_neg_mse': -2.0558085441589355,
 'loss': 6.19064474105835,
 'mean_acc': 0.0,
 'violence_neg_mse': -4.134836196899414}

# Archives

In [ ]:
# model3.load_weights('./bertsarct2.h5')

In [ ]:
# model3.summary()

In [ ]:
# history1=transfermodel.fit([ip1,ip2,ip3],y1,epochs=10,batch_size=64,callbacks=[callback1],validation_data=([ip1t,ip2t,ip3t],y_test1))

In [ ]:
# model3.load_weights('./bertsarcw2.h5')

In [ ]:
# plt.plot(history1.history['loss'])

In [ ]:
# plt.figure()
# plt.plot(history1.history['loss'])
# plt.plot(history1.history['val_loss'])
# # plt.savefig('rudditbertloss.png')

In [ ]:
# transfermodel.load_weights('./bertrudditw2.h5')

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
summarization2=[]
for i in range(len(df_language)):
  Article=str(df_language['Subtitles'][i][:1024])
  inputs=tokenizer([Article],return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=9023, early_stopping=True) # truncating = True
  summary=[tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  summarization2.append(summary)

In [ ]:
summarization2=pd.Series(summarization2)
summarization2